# Homework 5: Applying Functions and Iteration

Please complete this notebook by filling in the cells provided. Before you begin, execute the following cell. 

**Throughout this homework and all future ones, please be sure to not re-assign variables throughout the notebook! For example, if you use `max_temperature` in your answer to one question, do not reassign it later on. Moreover, please be sure to only put your written answers in the provided cells.**

In [1]:
# Don't change this cell; just run it. 

import numpy as np
from datascience import *

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

## 1. 2019 UNC Football Season


You will analyze how well the UNC football team performed in the 2019 season. A football game is divided into four quarters. If the game is tied after the 4th quarter, the game goes into overtime. The number of points UNC scored in each quarter and overtime (if needed), and the number of points their opponent scored in each quarter and overtime (if needed) are stored in a table called `UNC_fb.csv`.

In [4]:
# Just run this cell
# Read in the UNC_fb csv file
games = Table().read_table("UNC_fb.csv")
games.show()

Opponent,UNC 1Q,UNC 2Q,UNC 3Q,UNC 4Q,UNC OT,Opp 1Q,Opp 2Q,Opp 3Q,Opp 4Q,Opp OT
South Carolina,3,3,3,15,0,10,3,7,0,0
Miami FL,17,0,3,8,0,3,10,6,6,0
Wake Forest,0,0,3,15,0,7,14,0,3,0
App State,7,10,7,7,0,13,14,7,0,0
Clemson,7,7,0,6,0,0,14,0,7,0
Georgia Tech,3,14,0,21,0,0,0,7,15,0
Virgina Tech,10,7,7,7,10,7,14,0,10,12
Duke,7,0,10,3,0,3,0,14,0,0
Virginia,3,14,14,0,0,7,10,21,0,0
Pittsburgh,7,3,0,17,0,3,14,7,3,7


Let's start by finding the total points each team scored in a game.

**Question 1.** Write a function called `sum_scores`.  It should take five arguments, where each argument is the team's score for that quarter or overtime. It should return the team's total score for that game.


<!--
BEGIN QUESTION
name: q1_1
manual: false
-->

In [2]:
def sum_scores(Q1, Q2, Q3, Q4, OT):
    '''Returns the total score calculated by adding up the score of each quarter and overtime'''
    return Q1 + Q2 + Q3 + Q4 + OT

sum_scores(14, 7, 3, 0, 0) #DO NOT CHANGE THIS LINE

24

**Question 2.** Create a new table `final_scores` with three columns in this *specific* order: `Opponent`, `UNC Score`, `Opponent Score`. You will have to create the `UNC Score` and `Opponent Score` columns. Use the function `sum_scores` you just defined in the previous question for this problem.

*Hint:* If you want to apply a function that takes in multiple arguments, you can pass multiple column names as arguments in `tbl.apply()`. The column values will be passed into the corresponding arguments of the function.

*Tip:* If you’re running into issues creating final_scores, check that `UNC_scores` and `opponent_scores` output what you want. 


<!--
BEGIN QUESTION
name: q1_2
manual: false
-->

In [12]:
UNC_scores = sum_scores(games.column('UNC 1Q'), games.column('UNC 2Q'), games.column('UNC 3Q'), games.column('UNC 4Q'), games.column('UNC OT'))
opponent_scores = sum_scores(games.column('Opp 1Q'), games.column('Opp 2Q'), games.column('Opp 3Q'), games.column('Opp 4Q'), games.column('Opp OT'))
final_scores = Table().with_columns(
    'Opponent', games.column('Opponent'),
    'UNC Score', UNC_scores,
    'Opponent Score', opponent_scores
)
final_scores

Opponent,UNC Score,Opponent Score
South Carolina,24,20
Miami FL,28,25
Wake Forest,18,24
App State,31,34
Clemson,20,21
Georgia Tech,38,22
Virgina Tech,41,43
Duke,20,17
Virginia,31,38
Pittsburgh,27,34


We can get specific row objects from a table. You can use `tbl.row(n)` to get the `n`th row of a table. `row.item("column_name")` will allow you to select the element that corresponds to `column_name` in a particular row. Here's an example:

In [9]:
# Just run this cell
games.row(10)

Row(Opponent='Mercer', UNC 1Q=21, UNC 2Q=21, UNC 3Q=14, UNC 4Q=0, UNC OT=0, Opp 1Q=0, Opp 2Q=0, Opp 3Q=0, Opp 4Q=7, Opp OT=0)

In [10]:
# Just run this cell
games.row(10).item("UNC 1Q")

21

**Question 3.** We want to see for a particular game whether or not UNC won. Write a function called `did_UNC_win`.  It should take one argument: a row object from the `final_scores` table. It should return either `True` if UNC's score was greater than the Opponent's score, and `False` otherwise.


<!--
BEGIN QUESTION
name: q1_3
manual: false
-->

In [24]:
def did_UNC_win(row):
    if row.item('UNC Score') > row.item('Opponent Score'):
        return True
    else:
        return False

**Question 4.** You want to see how UNC did against every opponent during the 2019 season. Using the `final_scores` table, assign `results` to an array of `True` and `False` values that correspond to whether or not UNC won. Add the `results` array to the `final_scores` table, and assign this to `final_scores_with_results`. Then, respectively assign the number of wins and losses UNC had to `UNC_wins` and `UNC_losses`.

*Hint*: When you only pass a function name and no column labels through `tbl.apply()`, the function gets applied to every row in `tbl`


<!--
BEGIN QUESTION
name: q1_4
manual: false
-->

In [25]:
results = final_scores.apply(did_UNC_win)
final_scores_with_results = final_scores.with_column('results', results)
UNC_wins = final_scores_with_results.where('results', True).num_rows
UNC_losses = final_scores_with_results.where('results', False).num_rows

# Don't delete or edit the following line:
print(f"In the 2019 Season, UNC won {UNC_wins} games and lost {UNC_losses} games. Go Tar Heels!")

In the 2019 Season, UNC won 7 games and lost 6 games. Go Tar Heels!


## 2. Unrolling Loops

"Unrolling" a `for` loop means to manually write out all the code that it executes.  The result is code that does the same thing as the loop, but without the structure of the loop.  For example, for the following loop:

    for num in np.arange(3):
        print("The number is", num)

The unrolled version would look like this:

    print("The number is", 0)
    print("The number is", 1)
    print("The number is", 2)


Unrolling a `for` loop is a great way to understand what the loop is doing during each step. In this exercise, you'll practice unrolling `for` loops.


In each question below, write code that does the same thing as the given code, but with any `for` loops unrolled.  It's a good idea to run both your answer and the original code to verify that they do the same thing.  (Of course, if the code does something random, you'll get a different random outcome than the original code!)

First, run the cell below to load data that will be used in a few questions.  It's a table with 52 rows, one for each type of card in a deck of playing cards.  A playing card has a "suit" ("♠︎", "♣︎", "♥︎", or "♦︎") and a "rank" (2 through 10, J, Q, K, or A).  There are 4 suits and 13 ranks, so there are $4 \times 13 = 52$ different cards.

In [18]:
deck = Table.read_table("deck.csv")
deck

Rank,Suit
2,♠︎
2,♣︎
2,♥︎
2,♦︎
3,♠︎
3,♣︎
3,♥︎
3,♦︎
4,♠︎
4,♣︎


**Question 1.** Unroll the code below.

*Hint:* `np.random.randint` returns a random integer between 0 (inclusive) and the value that's passed in (exclusive).

<!--
BEGIN QUESTION
name: q2_1
manual: false
-->

In [28]:
# This table will hold the cards in a randomly-drawn hand of
# 5 cards.  We simulate cards being drawn as follows: We draw
# a card at random from the deck, make a copy of it, put the
# copy in our hand, and put the card back in the deck.  That
# means we might draw the same card multiple times, which is
# different from a normal draw in most card games.
hand = Table().with_columns("Rank", make_array(), "Suit", make_array())
for suit in np.arange(5):
    card = deck.row(np.random.randint(deck.num_rows))
    hand = hand.with_row(card)
hand

Rank,Suit
5,♠︎
4,♣︎
6,♣︎
10,♦︎
8,♥︎


In [33]:
hand = Table().with_columns("Rank", make_array(), "Suit", make_array())
hand = hand.with_row(deck.row(np.random.randint(deck.num_rows)))
hand = hand.with_row(deck.row(np.random.randint(deck.num_rows)))
hand = hand.with_row(deck.row(np.random.randint(deck.num_rows)))
hand = hand.with_row(deck.row(np.random.randint(deck.num_rows)))
hand = hand.with_row(deck.row(np.random.randint(deck.num_rows)))
hand

Rank,Suit
A,♦︎
7,♦︎
8,♦︎
4,♦︎
5,♦︎


**Question 2.** Unroll the code below.
<!--
BEGIN QUESTION
name: q2_2
manual: false
-->

In [36]:
for joke_iteration in np.arange(4):
    print("Knock, knock.")
    print("Who's there?")
    print("Banana.")
    print("Banana who?")
print("Knock, knock.")
print("Who's there?")
print("Orange.")
print("Orange who?")
print("Orange you glad I didn't say banana?")

Knock, knock.
Who's there?
Banana.
Banana who?
Knock, knock.
Who's there?
Banana.
Banana who?
Knock, knock.
Who's there?
Banana.
Banana who?
Knock, knock.
Who's there?
Banana.
Banana who?
Knock, knock.
Who's there?
Orange.
Orange who?
Orange you glad I didn't say banana?


In [37]:
print("Knock, knock.")
print("Who's there?")
print("Banana.")
print("Banana who?")
print("Knock, knock.")
print("Who's there?")
print("Banana.")
print("Banana who?")
print("Knock, knock.")
print("Who's there?")
print("Banana.")
print("Banana who?")
print("Knock, knock.")
print("Who's there?")
print("Banana.")
print("Banana who?")
print("Knock, knock.")
print("Who's there?")
print("Orange.")
print("Orange who?")
print("Orange you glad I didn't say banana?")

Knock, knock.
Who's there?
Banana.
Banana who?
Knock, knock.
Who's there?
Banana.
Banana who?
Knock, knock.
Who's there?
Banana.
Banana who?
Knock, knock.
Who's there?
Banana.
Banana who?
Knock, knock.
Who's there?
Orange.
Orange who?
Orange you glad I didn't say banana?
